In [2]:
using LinearAlgebra

In [254]:

"""
Xh = optshrink1(Y::AbstractMatrix, r::Int)
Perform rank−r denoising of data matrix `Y` using the OptShrink method
by Prof. Nadakuditi in this May 2014 IEEE Tr. on Info. Theory paper:
http://doi.org/10.1109/TIT.2014.2311661
In:
− `Y` 2D array where `Y = X + noise` and goal is to estimate `X`
− `r` estimated rank of `X`
Out:
− `Xh` rank−`r` estimate of `X` using OptShrink weights for SVD components
This version works only if the size of `Y` is sufficiently small,
because it performs calculations involving arrays roughly of
`size(Y'*Y)` and `size(Y*Y')`, so neither dimension of `Y` can be large.
"""
# function optshrink1(Y::AbstractMatrix, r::Int)
#     (U, s, V)  = svd(Y)
#     still = Diagonal(s)
    
# #     sigmahats = Diagonal(s[1:r])
#     sigmahats = s[1:r]
#     Xnoise_est = Diagonal(s[r+1:end])
    
#     W =  Array{Float64}(undef, r)

#     for i in 1:r
        
#         #theta_hats = Dz(sigmahats[i],Xnoise_est)
#         theta_hats = sqrt(1/Dz(sigmahats[i],Xnoise_est))
#         dpz =  Dpz(sigmahats[i], Xnoise_est)
        
#         display(theta_hats^2)
#         display(dpz)
#         W[i] =  -2/ (theta_hats^2* dpz)
        
#     end 
    
#     S_opt = U[:,1:r] * Diagonal(vec(W)) * V[:, 1:r]'
#     return S_opt
    
# end 



optshrink1

In [309]:
function optshrink1(Y::AbstractMatrix, r::Int)
    (U, s, V)  = svd(Y)
    n = size(Y)[1]
    m = size(Y)[2]
    
    S_ = diagm(n-r, m-r, s[r+1:end])
    
    W =  Array{Float64}(undef, r)

    for i in 1:r
        d =  Dz(s[i], S_) 
        d_ = Dpz(s[i], S_)
        W[i] =  -2 * d / d_
        
    end 
    
   S_opt = U[:,1:r] * Diagonal(vec(W)) * V[:, 1:r]'
    
    return S_opt
    
end 

optshrink1 (generic function with 1 method)

In [234]:
# function D(z, X)
#     n = size(X)[1]
#     m = size(X)[2]
#     In = Diagonal(ones(n))
#     Im = Diagonal(ones(m))
    
#     A = 1/n * tr(z * (z * z * In - X * X')^-1)
#     B = 1/m * tr(z * (z * z * Im - X' * X)^-1)
#     return A*B
# end 

# function D_(z, X)
#     n = size(X)[1]
#     m = size(X)[2]
    
#     A = 1/n * tr(z * (z*z * I - X * X')^-1)
#     B = 1/m * tr(-2 * z^2 * (z^2 * I - X' * X)^-2 + (z^2 * I - X' * X )^-1) 
#     C = 1/m * tr(z * (z^2 * I - X' * X)^-1)
#     D = 1/n * tr(-2 * z^2 * (z^2 * I - X * X')^-2 + (z^2 * I - X * X' )^-1) 
#     return A * B + C * D
# end 

D_ (generic function with 1 method)

In [310]:
function Dz(z, X)
    n = size(X)[1]
    m = size(X)[2]
    In = Diagonal(ones(n))
    Im = Diagonal(ones(m))
    z2IXXt = z^2*In - X*X';
    z2IXtX = z^2*Im - X'*X;
    invz2XtX = inv(z2IXtX);
    invz2XXt = inv(z2IXXt);

    D1z = 1/n*tr(z*invz2XXt);
    D2z = 1/m*tr(z*invz2XtX);

    Dz = D1z*D2z;

    return Dz
end 

function Dpz(z, X)
    n = size(X)[1]
    m = size(X)[2]
    In = Diagonal(ones(n))
    Im = Diagonal(ones(m))
    
    z2IXXt = z^2*In - X*X';
    z2IXtX = z^2*Im - X'*X;
    invz2XtX = inv(z2IXtX);
    invz2XXt = inv(z2IXXt);

    D1z = 1/n*tr(z*invz2XXt);
    D2z = 1/m*tr(z*invz2XtX);


    D1zp = 1/n*tr(-2*z^2*invz2XXt^2+invz2XXt);
    D2zp = 1/m*tr(-2*z^2*invz2XtX^2+invz2XtX);

    Dpz = D1z*D2zp+D1zp*D2z;
    return Dpz
end 

Dpz (generic function with 1 method)

In [293]:
using Random: seed!
seed!(0)
Y = rand(5,3)
display(Y)

Yh = optshrink1(Y, 2)
display(Yh)
    
   

#Yh = optshrink1(Y, 2)

5×3 Matrix{Float64}:
 0.823648  0.203477   0.585812
 0.910357  0.0423017  0.539289
 0.164566  0.0682693  0.260036
 0.177329  0.361828   0.910047
 0.27888   0.973216   0.167036

5×3 Matrix{Float64}:
 0.598335  0.236597   0.56591
 0.609996  0.154529   0.556546
 0.176241  0.0837109  0.169988
 0.448318  0.332532   0.46055
 0.237647  0.519169   0.324806

3

In [298]:
# include("optshrink1.jl") # uncomment if you need this
using Random: seed!
using LinearAlgebra: norm
seed!(0)
X = randn(30) * randn(20)' # outer product
Y = X + 40 * randn(size(X))
Xh_opt = optshrink1(Y, 1)
# Xh_lr = # you finish this to make the conventional rank−1 approximation
(U, s, V) =svd(Y)
Xh_lr = s[1]* U[:,1]  * V[:, 1]'

@show norm(Xh_opt - X)
@show norm(Xh_lr - X)

norm(Xh_opt - X) = 131.74000698480566
norm(Xh_lr - X) = 403.0278595936427


403.0278595936427

In [303]:
############
#P5
"""
Xh = optshrink2(Y::AbstractMatrix, r::Int)
Perform rank−`r` denoising of data matrix `Y` using the OptShrink method
by Prof. Nadakuditi in this May 2014 IEEE Tr. on Info. Theory paper:
http://doi.org/10.1109/TIT.2014.2311661
In:
− `Y` 2D array where `Y = X + noise` and goal is to estimate `X`
− `r` estimated rank of `X`
Out:
− `Xh` rank−`r` estimate of `X` using OptShrink weights for SVD components
This version works even if one of the dimensions of `Y` is large,
as long as the other is sufficiently small.
"""
function optshrink2(Y::AbstractMatrix, r::Int)
    (U, s, V)  = svd(Y)
    n = size(Y)[1]
    m = size(Y)[2]
    
    S_ = diagm(n-r, m-r, s[r+1:end])
    
    
    W =  Array{Float64}(undef, r)

    for i in 1:r
        d =  Dz_2(s[i], S_) 
        d_ = Dpz_2(s[i], S_)
        W[i] =  -2 * d / d_
        
    end 
    
   S_opt = U[:,1:r] * Diagonal(vec(W)) * V[:, 1:r]'
    
    return S_opt
    
end 

optshrink2

In [320]:
function Dz_2(z, S)
    K = size(S)[1]
    L = size(S)[2]
   
    A = 1/ (K*L)
    
    b = 0
    for i in 1:L
        b = b + z/(z*z - S[i][i]* S[i][i])
    end 
    B = (K - L) /z  + b 
    
    C = b
    

    return A * B * C 
end 

# function Dpz_2(z, S)
#     K = size(S)[1]
#     L = size(S)[2]
    
#     A1 = (L -K)/ (K*L)
#     A2 = 0 
    
#     for i in 1:L
#         A2 = A2 +  (2 * z) / ( z*z - S[i][i]*S[i][i])^2
#     end
    
#     A = A1 * A2
    
#     B1 = 1/(K*L)
#     B2 = 0
#     for i in 1:L
#         B2 = B2 + 2 * z * ( S[i][i] * S[i][i] + z*z) / (z*z - S[i][i] * S[i][i])^3
#     end
    
#     B2 = B2 *2
#     B = B1 * B2 

    
#     return A - B

function Dpz_2(z, S)
    K = size(S)[1]
    L = size(S)[2]
   
    A1 = 1 / K 
    A2 = 0
    A4= 0 
    for i in 1:L
        A2 = A2 + z / (z*z - S[i][i]*S[i][i])
        A4 = A4 +  (z*z + S[i][i]*S[i][i]) / ( z*z - S[i][i]*S[i][i])^2
    end
    B2 = A2
    A2 = A2 + (K - L)/z
    A3 = 1 /L 
    
    B4 = A4
    A = A1 * A2 * A3 * (-A4)
    
    
    
    B1 = 1/ L
    B3 = 1 / K
    B4 = -B4 - (K - L )/ (z*z)
    B =B1 * B2 *B3*B4
        
    return A + B
end 

Dpz_2 (generic function with 1 method)

In [321]:
using Random: seed!
seed!(0)
Y = rand(5,3)
display(Y)

Yh1 = optshrink1(Y, 2)
display(Yh1)
    
Yh2 = optshrink2(Y, 2)
display(Yh2)

5×3 Matrix{Float64}:
 0.823648  0.203477   0.585812
 0.910357  0.0423017  0.539289
 0.164566  0.0682693  0.260036
 0.177329  0.361828   0.910047
 0.27888   0.973216   0.167036

5×3 Matrix{Float64}:
 0.598335  0.236597   0.56591
 0.609996  0.154529   0.556546
 0.176241  0.0837109  0.169988
 0.448318  0.332532   0.46055
 0.237647  0.519169   0.324806

5×3 Matrix{Float64}:
 0.598335  0.236597   0.56591
 0.609996  0.154529   0.556546
 0.176241  0.0837109  0.169988
 0.448318  0.332532   0.46055
 0.237647  0.519169   0.324806

In [322]:
####提交版本

function optshrink2(Y::AbstractMatrix, r::Int)
    (U, s, V) = svd(Y)
    m = size(Y)[1]
    n = size(Y)[2]
    
    r = min(r, m, n)
    
    S = s[(r+1):end]
    
    w = zeros(r)
    for k = 1:r
        (D, Dder) = D_transform_from_matrix(s[k], S, max(m,n)-r, min(m,n)-r)
        w[k] = -2*D/Dder
    end
    
    Xh = U[:,1:r] * diagm(w) * V[:,1:r]'
    return Xh
end

function D_transform_from_matrix(z, sn, m, n)
    sm = [sn; zeros(m-n)]
    D11 = 1 ./ ((z^2) .- (sn.^2))
    D22 = 1 ./ ((z^2) .- (sm.^2))
    
    D1 = (1/n) * sum(z .* D11)
    D2 = (1/m) * sum(z .* D22)
    
    D = D1*D2
    
    D1_derivative = (1/n) * sum(-2 * (z^2) .* (D11.^2) + D11)
    D2_derivative = (1/m) * sum(-2 * (z^2) .* (D22.^2) + D22)
    
    D_der = D1 * D2_derivative + D2 * D1_derivative
   
    return (D, D_der)
end

D_transform_from_matrix (generic function with 1 method)

In [324]:
using LinearAlgebra: norm
using Random: seed!
seed!(0)
X = randn(10^5) * randn(100)' / 8 # test large case now
Y = X + randn(size(X))
Xh_opt = optshrink2(Y, 1)
# Xh_lr = # you finish this part
(U, s, V) =svd(Y)
Xh_lr = s[1]* U[:,1]  * V[:, 1]'

@show norm(Xh_opt - X)
@show norm(Xh_lr - X)

norm(Xh_opt - X) = 240.35441054181598
norm(Xh_lr - X) = 317.08494230558097


317.08494230558097